Imports

In [28]:
import sys
import warnings
from pathlib import Path
import numpy as np
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from diffpy.utils.parsers.loaddata import loadData
from bg_mpl_stylesheet.bg_mpl_stylesheet import bg_mpl_style

In [29]:
warnings.filterwarnings("ignore", category=UserWarning)

Voltage label to use when plotting.

In [30]:
D_ELABELS = dict(li="$E_{\mathrm{we}}\;\mathrm{vs.\;Li/Li^{+}\;[V]}$",
                 la="$E_{\mathrm{we}}\;\mathrm{vs.\;Na/Na^{+}\;[V]}$",
                 general="$V\;[\mathrm{V}]$",
                 )
ELABEL = D_ELABELS["li"]

State of charge label for echem and difference between them.

In [31]:
XLABEL = "$x\;\mathrm{in}\;\mathrm{Li}_{x}\mathrm{TiO_{2}}$"
X_LABEL_DIFF = 0.2

Stating the file name of the iPython notebook.

In [32]:
nb_name = "xy_overview_echem_t_x_v.ipynb"

Dictionary with plot settings.

In [33]:
D_PLOT = dict(dpi=600,
              figsize_xy=(8, 4),
              figsize_echem=(8, 4),
              figsize_xy_echem=(8, 8),
              fs_labels=20,
              fs_ticks=14,
              xmin=1,
              xmax=10,
              emin=1,
              emax=3,
              tlabel="$t\;[\mathrm{h}]$",
              elabel=ELABEL,
              xlabel=XLABEL,
              )

Printing the dictionary with plot settings.

In [34]:
print(f"{80*'-'}\nThe following plot settings were found in the top of the "
      f"{nb_name} file:")
for k in list(D_PLOT.keys()):
      if len(k) < 8:
            print(f"\t{k}\t\t\t{D_PLOT[k]}")
      elif 8 <= len(k) < 16:
            print(f"\t{k}\t\t{D_PLOT[k]}")
      else:
            print(f"\t{k}\t{D_PLOT[k]}")
print(f"{80*'-'}\nPlease change the settings and rerun the code if neccessary.")

--------------------------------------------------------------------------------
The following plot settings were found in the top of the xy_overview_echem_t_x_v.ipynb file:
	dpi			600
	figsize_xy		(8, 4)
	figsize_echem		(8, 4)
	figsize_xy_echem	(8, 8)
	fs_labels		20
	fs_ticks		14
	xmin			1
	xmax			10
	emin			1
	emax			3
	tlabel			$t\;[\mathrm{h}]$
	elabel			$E_{\mathrm{we}}\;\mathrm{vs.\;Li/Li^{+}\;[V]}$
	xlabel			$x\;\mathrm{in}\;\mathrm{Li}_{x}\mathrm{TiO_{2}}$
--------------------------------------------------------------------------------
Please change the settings and rerun the code if neccessary.


Checking whether `data_xy` folder for data files exists.

In [35]:
data_xy_path = Path.cwd() / "data_xy"
if not data_xy_path.exists():
    data_xy_path.mkdir()
    s = f"\n{80*'-'}\nA folder called '{data_xy_path.name}' has been created.\n"
    s += f"Please put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

Checking whether `data_echem` folder exists.

In [36]:
data_echem_path = Path.cwd() / "data_echem"
if not data_echem_path.exists():
    data_echem_path.mkdir()
    s = f"\n{80*'-'}\nA folder called '{data_echem_path.name}' has been "
    s += f"created.\nPlease put your data files there and rerun the code."
    s += f"\n{80*'-'}"
    sys.exit(s)

Checking whether any data files are present in the `data_xy` folder.

In [37]:
data_xy_files = list(data_xy_path.glob("*.*"))
if len(data_xy_files) == 0:
    s = f"\n{80*'-'}\nNo files were found in the '{data_xy_path.name}' folder."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

Checking whether that a single file is present in the `data_echem` folder.

In [38]:
data_echem_files = list(data_echem_path.glob("*.*"))
if len(data_echem_files) == 0:
    s = f"\n{80*'-'}\nNo files were found in the '{data_echem_path.name}' "
    s += f"folder.\nPlease put your data file there and rerun the code."
    s += f"\n{80*'-'}"
    sys.exit(s)
elif len(data_echem_files) > 1:
    s = f"\n{80*'-'}\nMore than one file was found in the "
    s += f"'{data_echem_path.name}' folder.\nPlease put your single data file "
    s += f"there and rerun the code.\n{80*'-'}"
    sys.exit(s)
else:
    data_echem_file = data_echem_files[0]

Checking that only one file extension is present among the data_xy files in the 
'data' folder.

In [39]:
suffixes = []
for f in data_xy_files:
    if f.suffix not in suffixes:
        suffixes.append(f.suffix)
if len(suffixes) > 1:
    print(f"{80*'-'}\nThe following file extensions were found in the "
          f"'{data_xy_path.name}' folder:")
    for suffix in suffixes:
        print(f"\t{suffix}")
    s = f"\n{80*'-'}\nPlease review the '{data_xy_path.name}' folder to ensure "
    s += f"that only one file extension is\npresent.\n{80*'-'}"
    sys.exit(s)
else:
    D_PLOT["suffix"] = suffixes[0].strip(".")

Updating dictionary with plot settings with labels according to the file 
extension of the data files.

In [40]:
if suffixes[0] == ".gr":
    D_PLOT["ylabel"] = "$r\;[\mathrm{\AA}]$"
    D_PLOT["cbarlabel"] = "$G\;[\mathrm{\AA}^{-2}]$"
elif suffixes[0] == ".fq":
    D_PLOT["ylabel"] = "$Q\;[\mathrm{\AA}^{-1}]$"
    D_PLOT["cbarlabel"] = "$F\;[\mathrm{\AA}^{-1}]$"
else:
    D_PLOT["ylabel"] = "$Q\;[\mathrm{\AA}^{-1}]$"
    D_PLOT["cbarlabel"] = "$I\;[\mathrm{arb.\;u.}]$"

Collecting x array from first data file and stacking y arrays into one array.

In [41]:
for i, f in enumerate(data_xy_files):
    data = loadData(f)
    x, y = data[:, 0], data[:, 1]
    if i == 0:
        array = y
    else:
        array = np.column_stack((array, y))
D_XY = dict(x=x, y=array)

In [42]:
def current_change_indices(current):
    indices = [i for i in range(1, len(current))
               if current[i] != 0 and current[i] * current[i-1] <= 0]
    
    return indices

In [43]:
def get_x_indices(xlabels, x):
    indices = []
    j = 0
    for e in xlabels:
        for i in range(j, len(x)):
            is_close = np.isclose(np.array(e, dtype=float), 
                                  x[i],
                                  atol=abs(x[0] - x[1]),
                                  )
            if is_close:
                indices.append(i)
                j = i + 1
                break
                
    return indices

In [44]:
def get_labels(x, indices, labeldiff):
    for i in range(len(indices)):
        if i == 0:
            if x[0] - x[1] < 0:
                sign = 1
            else:
                sign = -1
            x_labels_vals = np.arange(x[0], 
                                      x[indices][0], 
                                      sign * labeldiff,
                                      )
        else:
            if i % 2 != 0:
                x_labels_vals = np.append(x_labels_vals, 
                                          np.arange(x_labels_vals[-1],
                                                    x[indices[i]], 
                                                    - sign * labeldiff,
                                                    ))
                x_labels_vals = np.append(x_labels_vals, x[indices[i]])
            else:
                x_labels_vals = np.append(x_labels_vals, 
                                          np.arange(x_label_vals[-1],
                                                    x[indices[i]],
                                                    sign * labeldiff,
                                                    ))
    if indices[-1] != len(x) - 1:
        if x_labels_vals[-1] - x_labels_vals[-2] < 0:
            sign = 1
        else:
            sign = -1
        last, remainder = x_labels_vals[-1], x_labels_vals[-1] % labeldiff
        start_val =  last - remainder + labeldiff
        x_labels_vals = np.append(x_labels_vals,
                                  np.arange(start_val,
                                            x[-1],
                                            sign * labeldiff,
                                            ))
    x_labels = []
    for i in range(1, len(x_labels_vals)):
        is_close = np.isclose(x_labels_vals[i-1], 
                              x_labels_vals[i], 
                              atol=labeldiff / 2,
                              )
        if not is_close:
            if f"{x_labels_vals[i-1]:.2f}".split(".")[-1][1] == "0":
                x_labels.append(f"{x_labels_vals[i-1]:.1f}")
            else:
                x_labels.append(f"{x_labels_vals[i-1]:.2f}")
        if i == len(x_labels_vals) - 1:
            if f"{x_labels_vals[-1]:.2f}".split(".")[-1][1] == "0":
                x_labels.append(f"{x_labels_vals[-1]:.1f}")
            else:
                x_labels.append(f"{x_labels_vals[-1]:.2f}")

    return x_labels

Collecting time and voltage arrays for echem file. (From first and second 
column in the file.)

In [45]:
data = np.loadtxt(data_echem_file)
time, voltage, current, x_ec = data.T
change_indices = current_change_indices(current)
xlabels = get_labels(x_ec, change_indices, X_LABEL_DIFF)
xlabel_indices = get_x_indices(xlabels, x_ec)
time_xlabels = [time[e] for e in xlabel_indices]
D_EC = dict(time=time, 
            voltage=voltage, 
            x_ec=x_ec, 
            xlabels=xlabels, 
            time_xlabels=time_xlabels,
            )

Function to obtain indices for min and max values of array.

In [46]:
def get_indices(array, min_value, max_value):
    min_index, max_index = None, None
    for i, e in enumerate(array):
        if e >= min_value:
            min_index = i
            break
    for i, e in enumerate(array):
        if e >= max_value:
            max_index = i
            break
    if isinstance(min_index, type(None)):
        min_index = 0
    if isinstance(max_index, type(None)):
        max_index = len(array) - 1

    return min_index, max_index

Shaping x array and stacked y array.

In [47]:
xmin_index, xmax_index = get_indices(x, D_PLOT["xmin"], D_PLOT["xmax"])
if xmax_index < len(x) - 1:
    x, array = x[xmin_index:xmax_index + 1], array[xmin_index:xmax_index + 1, :]
else:
    x, array = x[xmin_index:], array[xmin_index:, :]
D_PLOT["vmin"], D_PLOT["vmax"] = np.amin(array), np.amax(array)

Function for shifting (diverging) colormap, such thast white corresponds to zero
even if data is min and max is not symmetric around zero, i.e., if min != - max.

In [48]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False),
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = mcolors.LinearSegmentedColormap(name, cdict)
    mpl.colormaps.register(cmap=newcmap, force=True)

    return newcmap

Function to truncate (diverging) colormap to get the positive part.

In [49]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
               'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name,
                                                   a=minval,
                                                   b=maxval
                                                   ),
               cmap(np.linspace(minval, maxval, n)))

    return new_cmap

Creating diverging colormap with user-defined colors, shrinking colormap to 
adapt to data min and max values to have white corresponding to zero, and 
truncating the diverging colormap to get its positive part.

In [50]:
div_gradient = mcolors.LinearSegmentedColormap.from_list('div_gradient', (
                 # Edit this gradient at 
                 # https://eltos.github.io/gradient/#0B3C5D-0B3C5D-FFFFFF-B82601-B82601
                 (0.000, (0.043, 0.235, 0.365)),
                 (0.250, (0.200, 0.400, 0.500)),
                 (0.500, (1.000, 1.000, 1.000)),
                 (0.750, (0.850, 0.200, 0.100)),
                 (1.000, (0.722, 0.149, 0.004))))
mp = 1 - D_PLOT["vmax"] / (D_PLOT["vmax"] + abs(D_PLOT["vmin"]))
cmap_shrunk = shiftedColorMap(div_gradient, 
                              start=0, 
                              midpoint=mp, 
                              stop=1, 
                              name='shrunk',
                              )
cmap_trunc = truncate_colormap(div_gradient, 0.5, 1)                              

Updating dictionary with plot settings to include colormap.

In [51]:
if suffixes[0] == ".gr":
    D_PLOT["cmap"] = cmap_shrunk
elif suffixes[0] == ".fq":
    D_PLOT["cmap"] = cmap_shrunk
else:
    D_PLOT["cmap"] = cmap_trunc   

Plot function for overview plot.

In [52]:
def plot_xy(x, array, d, output_paths):
    plt.style.use(bg_mpl_style)
    fig, ax = plt.subplots(figsize=d["figsize_xy"])
    im = ax.imshow(array,
                   interpolation="none",
                   aspect="auto",
                   extent=(0, array.shape[1], d["xmax"], d["xmin"]),
                   vmin=np.amin(array),
                   vmax=np.amax(array),
                   cmap=d["cmap"],
                   )
    ax.tick_params(axis="x",
                   top=True,
                   bottom=True,
                   labeltop=True,
                   labelbottom=False,
                   )
    ax.tick_params(axis="y",
                   left=True,
                   right=True,
                   labelleft=True,
                   labelright=False,
                   )                   
    ax.set_xlabel(d["xlabel"], fontsize=d["fs_labels"])
    ax.xaxis.set_label_position("top")
    ax.set_ylabel(ylabel=d["ylabel"], fontsize=d["fs_labels"])
    ax.minorticks_on()
    cbar = plt.colorbar(im)
    cbar.set_label(label=d["cbarlabel"], fontsize=d["fs_labels"])
    outputname = f"{d['suffix']}_overview_xmin={d['xmin']}_xmax={d['xmax']}"
    for p in output_paths:
        print(f"\t\t{p.name}")
        plt.savefig(p / f"{outputname}.{p.name}", 
                    bbox_inches="tight",
                    dpi=d["dpi"],
                    )
    plt.close()

    return None

Plot function for echem.

In [53]:
def plot_echem(time, voltage, d, output_paths):
    plt.style.use(bg_mpl_style)
    fig, ax = plt.subplots(figsize=d["figsize_echem"])
    ax.plot(time, voltage)
    ax.set_xlabel(d["tlabel"], fontsize=d["fs_labels"])
    ax.set_ylabel(d["elabel"], fontsize=d["fs_labels"])
    ax.set_xlim(np.amin(time), np.amax(time))
    ax.set_ylim(d["emin"], d["emax"])
    ax.tick_params(axis="both", labelsize=d["fs_ticks"])                    
    ax.minorticks_on()
    for p in output_paths:
        print(f"\t\t{p.name}")
        plt.savefig(p / f"echem_plot.{p.name}", 
                    bbox_inches="tight",
                    dpi=d["dpi"],
                    )
    plt.close()

    return None

Plot function for xy overview and echem plot.

In [54]:
def plot_xy_echem(d_xy, d_ec, d, output_paths):
    x, y = d_xy["x"], d_xy["y"]
    time, voltage = d_ec["time"], d_ec["voltage"]
    x_ec = d_ec["x_ec"]
    xlabels, time_xlabels = d_ec["xlabels"], d_ec["time_xlabels"]
    time_min, time_max = np.amin(time), np.amax(time)
    plt.style.use(bg_mpl_style)
    fig = plt.figure(figsize=d["figsize_xy_echem"])
    grid = plt.GridSpec(nrows=2, 
                        ncols=2,
                        hspace=0.1,
                        height_ratios=(1, 0.5), 
                        width_ratios=(1, 0.135),
                        )
    ax1 = fig.add_subplot(grid[1,0])
    ax1.set_xlim(time_min, time_max)
    ax1.set_ylim(d["emin"], d["emax"])
    ax1.set_xlabel(d["xlabel"], fontsize=d["fs_labels"])
    ax1.set_ylabel(d["elabel"], fontsize=d["fs_labels"])
    ax11 = ax1.twiny()
    ax11.plot(time, voltage, zorder=0)
    ax11.set_xlim(time_min, time_max)
    ax11.set_ylim(d["emin"], d["emax"])
    ax11.tick_params(axis="x",
                     labeltop=False,
                     labelbottom=False,
                     labelsize=d["fs_ticks"],
                     )

    ax0 = fig.add_subplot(grid[0, 0:])
    im = ax0.imshow(array,
                   interpolation="none",
                   aspect="auto",
                   extent=(time_min, time_max, d["xmax"], d["xmin"]),
                   vmin=np.amin(array),
                   vmax=np.amax(array),
                   cmap=d["cmap"],
                   )
    ax0.tick_params(axis="x",
                   top=True,
                   bottom=True,
                   labeltop=True,
                   labelbottom=False,
                   )
    ax0.tick_params(axis="y",
                   left=True,
                   right=True,
                   labelleft=True,
                   labelright=False,
                   )                   
    ax0.set_xlabel(d["tlabel"], fontsize=d["fs_labels"])
    ax0.xaxis.set_label_position("top")
    ax0.set_ylabel(ylabel=d["ylabel"], fontsize=d["fs_labels"])
    ax0.minorticks_on()
    cbar = plt.colorbar(im)
    cbar.set_label(label=d["cbarlabel"], fontsize=d["fs_labels"])
    ax1.plot(time, voltage)
    ax1.set_xlabel(d["xlabel"], fontsize=d["fs_labels"])
    ax1.set_ylabel(d["elabel"], fontsize=d["fs_labels"])
    ax1.set_xlim(np.amin(time), np.amax(time))
    ax1.set_ylim(d["emin"], d["emax"])
    ax1.tick_params(axis="both", labelsize=d["fs_ticks"])
    ax1.set_xticks(time_xlabels)  
    ax1.set_xticklabels(xlabels)
    ax1.minorticks_on()
    outputname = f"{d['suffix']}_overview_echem"
    outputname += f"xmin={d['xmin']}_xmax={d['xmax']}"
    for p in output_paths:
        print(f"\t\t{p.name}")
        plt.savefig(p / f"{outputname}.{p.name}", 
                    bbox_inches="tight",
                    dpi=d["dpi"],
                    )
    plt.close()

    return None

Creating plot directories if not already existing.

In [55]:
plot_folders = ["png", "pdf", "svg"]
plot_paths = [Path.cwd() / folder for folder in plot_folders]
for p in plot_paths:
    if not p.exists():
        p.mkdir()

Plotting.

In [56]:
print(f"{80*'-'}\nPlotting...\n\txy overview...")
plot_xy(x, array, D_PLOT, plot_paths)
print(f"\techem...")
plot_echem(time, voltage, D_PLOT, plot_paths)
print(f"\txy overview w. echem...")
plot_xy_echem(D_XY, D_EC, D_PLOT, plot_paths)
print(f"Done.\n{80*'-'}\nPlease see the {plot_folders} folders.")

--------------------------------------------------------------------------------
Plotting...
	xy overview...
		png
		pdf
		svg
	echem...
		png
		pdf
		svg
	xy overview w. echem...
		png
		pdf
		svg
Done.
--------------------------------------------------------------------------------
Please see the ['png', 'pdf', 'svg'] folders.
